In [5]:
data = open('input.txt', 'r').read()


chars = list(set(data))
data_size, X_size = len(data), len(chars)
print("data has %d characters, %d unique" % (data_size, X_size))
char_to_idx = {ch:i for i,ch in enumerate(chars)}
idx_to_char = {i:ch for i,ch in enumerate(chars)}

data has 99993 characters, 62 unique


In [6]:
pointer = 0
seq_len = 20

In [7]:
inputs = ([char_to_idx[ch] 
           for ch in data[pointer: pointer + seq_len]])
targets = ([char_to_idx[ch] 
            for ch in data[pointer + 1: pointer + seq_len + 1]])

In [8]:
inputs

[9, 3, 30, 53, 29, 42, 52, 19, 19, 13, 42, 56, 19, 6, 53, 22, 2, 52, 53, 29]

In [9]:
targets

[3, 30, 53, 29, 42, 52, 19, 19, 13, 42, 56, 19, 6, 53, 22, 2, 52, 53, 29, 42]

In [10]:
def sequence_to_model_input(sequence, vocab_size):
    x = {}
    for t in range(len(sequence)):
        x[t] = np.zeros((1, vocab_size))
        x[t][0, inputs[t]] = 1
    return x

In [11]:
t = sequence_to_model_input(inputs, 62)

Model will output a 62 x 1 output. We _want_ it to output 0s for everything, but 1s for the correct target.

# Model definition

In [109]:
class LSTM_Model:
    '''
    An LSTM model with one LSTM layer that feeds data through it and generates an output.
    '''
    def __init__(self, sequence_length, vocab_size, hidden_size, learning_rate):
        '''
        Initialize list of nodes of length the sequence length
        List the vocab size and the hidden size 
        Initialize the params
        '''
        self.nodes = [LSTM_Node(hidden_size, vocab_size) for x in range(sequence_length)]
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.start_H = np.zeros(hidden_size)
        self.start_C = np.zeros(hidden_size)
        self.params = LSTM_Params(hidden_size, vocab_size)
        self.learning_rate = learning_rate
        self.forward_step = 0
        
    def forward(self, x_batch, first_iter=False):

        batch_size = x_batch.shape[0]
        x_out = x_batch
        if first_iter:
            h_in = np.zeros((batch_size, self.hidden_size))
            c_in = np.zeros((batch_size, self.hidden_size))
        else:
            h_in = self.nodes[0].H_out
            c_in = self.nodes[0].C_out            
        for node in self.nodes:
            x_out, h_in, c_in = node.forward(x_out, h_in, c_in, self.params)
        self.forward_step += 1
        return x_out

    
    def loss(self, prediction, y_batch):
        '''
        Return a gradient: how much our prediction influences how much we "missed" by.
        '''
        return (y_batch - prediction) ** 2

    
    def loss_gradient(self, prediction, y_batch):
        '''
        Return a gradient: how much our prediction influences how much we "missed" by.
        '''
        return -1.0 * (y_batch - prediction)
            
    def backward(self, loss_grad):
        '''
        Implements the "Backpropagation Through Time" algorithm.
        For each step in the sequence T moving backwards, backpropagate through some number
        K nodes
        
        '''
        # Initialize h_grad and c_grad
        # Initialize x_in, h_in, c_in, run em through
        batch_size = loss_grad.shape[0]
        H_grad = np.zeros((batch_size, self.hidden_size))
        C_grad = np.zeros((batch_size, self.hidden_size))
        Y_grad = loss_grad        
        T = self.sequence_length - 1
        K = 5 # BPTT length
        
        # BPTT
        num_iterations = T - K
        for n in range(num_iterations):
            for t in range(T-n, T-K-n, -1):
#                 import pdb; pdb.set_trace()
                Y_grad, H_grad, C_grad = \
                    self.nodes[t-n].backward(Y_grad, H_grad, C_grad, self.params)
        
#         self.start_H = H_grad
#         self.start_C = C_grad
        return 


    def single_step(self, x, y, first_iter):
        prediction = self.forward(x, first_iter=first_iter)
        loss_gradient = self.loss_gradient(x, y)
        self.backward(loss_gradient)
        self.params.update_params(self.learning_rate)

        
    def loss_batch(self, x, y):
        prediction = self.forward(x, first_iter=False)
        return np.sum(self.loss(prediction, y))

In [110]:
class LSTM_Node:
    '''
    An LSTM Node that takes in input and generates output. 
    Has a size of its hidden layers and a vocabulary size it expects.
    '''
    def __init__(self, hidden_size, vocab_size):
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        
    def forward(self, X_input, H_input, C_input, LSTM_params):

        self.Z = np.column_stack((X_input, H_input))

        # Step 2
        self.F_int = np.dot(self.Z, LSTM_params.W_F)
        self.F = sigmoid(self.F_int)

        # Step 3
        self.I_int = np.dot(self.Z, LSTM_params.W_I)
        self.I = sigmoid(self.I_int)

        # Step 4
        self.C_prop_int = np.dot(self.Z, LSTM_params.W_C)
        self.C_prop = tanh(self.C_prop_int)

        # Step 5 
        self.C_prop = tanh(self.C_prop_int)
#         import pdb; pdb.set_trace()
        self.C_out = self.F * C_input + self.I * self.C_prop

        # Step 6
        self.O_int = np.dot(self.Z, LSTM_params.W_O)
        self.O = sigmoid(self.O_int)

        # Step 7
        self.C_act = tanh(self.C_out)
        self.H_out = self.O * self.C_act

        # Step 8
        self.X_out = np.dot(self.H_out, LSTM_params.W_V)

        return self.X_out, self.H_out, self.C_out


    def backward(self, Y_grad, H_grad, C_grad, LSTM_params):
        # Initialize the gradient for the words and the hidden layers:
        self.Z_diff = np.zeros_like(self.Z)
        
        # 2
        LSTM_params.W_V_diff += np.dot(self.H_out.T, Y_grad)

        # 3
        self.H_diff = np.dot(Y_grad, LSTM_params.W_V_diff.T)
        self.H_diff += H_grad

        # 4
        self.O_diff = self.H_diff * self.C_act

        # 4.5
        self.O_int_diff = sigmoid(self.O, deriv=True) * self.O
        
        # 5
        LSTM_params.W_O_diff += np.dot(self.Z.T, self.O_int_diff)
        self.Z_diff = np.dot(self.O_int_diff, LSTM_params.W_O.T)
        
        # 6
        self.C_diff = C_grad
        self.C_diff += self.H_diff * self.O * tanh(self.C_act, deriv=True)

        # 7
        self.C_prop_diff = self.C_diff * self.I

        # 7.5
        self.C_prop_int_diff = tanh(self.C_prop, deriv = True) * self.C_prop_diff

        # 8
        LSTM_params.W_C_diff += np.dot(self.Z.T, self.C_prop_int_diff)
        self.Z_diff += np.dot(self.C_prop_int, LSTM_params.W_C.T)
        
        # 9
        self.I_diff = self.C_diff * self.C_prop

        # 9.5
        self.I_int_diff = sigmoid(self.I, deriv=True) * self.I_diff

        # 10
        LSTM_params.W_I_diff += np.dot(self.Z.T, self.I_int_diff)
        self.Z_diff += np.dot(self.I_int, LSTM_params.W_I.T)
        
        # 11
        self.F_diff = self.C_diff * self.C_out

        # 11.5
        self.F_int_diff = sigmoid(self.F, deriv=True) * self.F_diff

        # 12
        LSTM_params.W_F_diff += np.dot(self.Z.T, self.F_int)        
        self.Z_diff += np.dot(self.F_int, LSTM_params.W_F.T)
        
        # 13
        C_grad = self.F * self.C_diff

        # 14
        X_grad = self.Z[:, :self.vocab_size]
        H_grad = self.Z[:, self.vocab_size:,]        

        return X_grad, H_grad, C_grad

In [111]:
class LSTM_Params:
    
    def __init__(self, hidden_size, vocab_size):
        self.stack_size = hidden_size + vocab_size
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        
        self.W_F = np.random.normal(size=(self.stack_size, self.hidden_size))
        self.W_I = np.random.normal(size=(self.stack_size, self.hidden_size))
        self.W_C = np.random.normal(size=(self.stack_size, self.hidden_size))
        self.W_O = np.random.normal(size=(self.stack_size, self.hidden_size))
        self.W_V = np.random.normal(size=(self.hidden_size, self.vocab_size))
        
        self.W_F_diff = np.zeros_like(self.W_F)
        self.W_I_diff = np.zeros_like(self.W_I)
        self.W_C_diff = np.zeros_like(self.W_C)
        self.W_O_diff = np.zeros_like(self.W_O)
        self.W_V_diff = np.zeros_like(self.W_V)

    def update_params(self, learning_rate):
        self.W_F -= learning_rate * self.W_F_diff
        self.W_I -= learning_rate * self.W_I_diff
        self.W_C -= learning_rate * self.W_C_diff
        self.W_O -= learning_rate * self.W_O_diff
        self.W_V -= learning_rate * self.W_V_diff

# Activations

In [112]:
def sigmoid(x, deriv=False):
    if deriv:
        return sigmoid(x) * (1 - sigmoid(x))
    else:
        return 1 / (1 + np.exp(-x))

def tanh(x, deriv=False):
    if deriv:
        return 1 - x * x
    else:
        return np.tanh(x)

# Initialization

In [113]:
mod = LSTM_Model(sequence_length=20, vocab_size=62, hidden_size=100, learning_rate=0.1)

## Data

In [114]:
inputs

[11, 49, 50, 30, 39, 61, 10, 36, 36, 20, 61, 47, 36, 8, 30, 35, 1, 10, 30, 39]

In [115]:
targets

[11, 49, 50, 30, 39, 61, 10, 36, 36, 20, 61, 47, 36, 8, 30, 35, 1, 10, 30, 39]

In [125]:
class Character_generator:
    
    def __init__(self, text_data, model):
        self.data = text_data
        self.model = model
        self.chars = list(set(data))
        self.vocab_size = len(self.chars)
        self.char_to_idx = {ch:i for i,ch in enumerate(self.chars)}
        self.idx_to_char = {i:ch for i,ch in enumerate(self.chars)}
        
    def generate_sequences(self, start_pos, seq_length):
        input_sequence = ([self.char_to_idx[ch] 
                           for ch in self.data[start_pos:start_pos + seq_length]])
        target_sequence = ([self.char_to_idx[ch] 
                            for ch in self.data[start_pos:start_pos + seq_length]])
        return input_sequence, target_sequence

    def sequence_to_model_input(self, sequence, vocab_size):
        out_batch = np.zeros((len(sequence), vocab_size))
        for i, el in enumerate(sequence):
            out_batch[i, el] = 1        
        return out_batch
    
    def generate_batch(self, start_pos):
        input_sequence, target_sequence = self.generate_sequences(start_pos, self.model.sequence_length)
        return self.sequence_to_model_input(input_sequence, self.vocab_size), \
            self.sequence_to_model_input(target_sequence, self.vocab_size) 
    
    def train(self, steps):
        start_pos = 0
        iterations = 0
        while iterations < steps:
            x_batch, y_batch = self.generate_batch(start_pos)
            first_iter = True if iterations == 0 else False
            self.model.single_step(x_batch, y_batch, first_iter)
            if iterations % 100 == 0:
                print("Loss", self.model.loss_batch(x_batch, y_batch))
            
            start_pos += self.model.sequence_length
            iterations += 1

In [126]:
data = open('input.txt', 'r').read()
mod = LSTM_Model(sequence_length=20, vocab_size=62, hidden_size=100, learning_rate=0.00001)
character_generator = Character_generator(data, mod)
character_generator.train(2000, check_every=100)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in exp
  """


Loss 18533.891628121077
Loss 3598.445927655147
Loss 11814.897280359244
Loss 1002273.0225738513


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:97: RuntimeWarning: overflow encountered in add


KeyboardInterrupt: 

TODO: 

* Explore other optimization algorithms other than straightforward SGD. 
* Use gradient clipping
